# Use pytorch in distance


In [3]:
import numpy as np
import torch

samples = []
from scipy.stats import multivariate_normal
for i in range(784):
    x = np.random.rand(19)
    y = multivariate_normal.pdf(x, mean=2.5, cov=0.5);

    samples.append(y)



In [4]:
from torch import nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 128, 64, 10 units each
        self.fc1 = nn.Linear(19, 128)
        self.fc2 = nn.Linear(128, 64)
        # Output layer, 10 units - one for each digit
        self.fc3 = nn.Linear(64, 1)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
#         x = F.softmax(x, dim=1)
        
        return x
    
model = Network()


In [5]:
from torch import optim

# https://pytorch.org/docs/stable/nn.html
criterion = nn.L1Loss()


optimizer = optim.SGD(model.parameters(), lr=0.003)
# criterion = nn.CrossEntropyLoss()


In [7]:
epochs = 20
## initial dataset
initial = np.random.rand(19)
sample_ = torch.from_numpy(initial).float()
output_ = model(sample_).data
print (output_)

for e in range(epochs):
    running_loss = 0
    for sample in samples:
        sample = torch.from_numpy(sample).float()
        optimizer.zero_grad()
        output = model(sample)
        loss = criterion(output, output_)
#         loss = torch.div(output, output_, out=None)
        loss.backward()
        optimizer.step()
        output_ = output.data
        
        running_loss += loss.item()
    else:
        print("Training loss: " + str(running_loss/len(samples)))

tensor([0.0472])
Training loss: 0.004129091619837041
Training loss: 0.004161949754141423
Training loss: 0.004094392251299352
Training loss: 0.004000822737889022
Training loss: 0.004109085917624892
Training loss: 0.003958505293240352
Training loss: 0.0040169617123141585
Training loss: 0.003993554173835686
Training loss: 0.003971532766460156
Training loss: 0.0039765391574830425
Training loss: 0.004004289244054531
Training loss: 0.003984496054448643
Training loss: 0.004069297061282761
Training loss: 0.003946385631451802
Training loss: 0.0039278197539400084
Training loss: 0.004013137226658208
Training loss: 0.003893345252287631
Training loss: 0.003877510924880602
Training loss: 0.003862112969616238
Training loss: 0.003872616273560086
